In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from experiment import experiment

In [3]:
import os

In [4]:
import pandas as pd

In [5]:
results = os.listdir('results')

In [6]:
results = [result for result in results if result.endswith('.pkl') and "5.31.2023" in result and "nn" in result and "std" in result]

In [7]:
results

['star_nn_experiment_std_05.31.2023.pkl',
 'concrete_nn_experiment_std_05.31.2023.pkl',
 'meps_21_nn_experiment_std_05.31.2023.pkl',
 'facebook_2_nn_experiment_std_05.31.2023.pkl',
 'bio_nn_experiment_std_05.31.2023.pkl',
 'blog_data_nn_experiment_std_05.31.2023.pkl',
 'meps_19_nn_experiment_std_05.31.2023.pkl',
 'meps_20_nn_experiment_std_05.31.2023.pkl',
 'community_nn_experiment_std_05.31.2023.pkl',
 'homes_nn_experiment_std_05.31.2023.pkl',
 'facebook_1_nn_experiment_std_05.31.2023.pkl',
 'bike_nn_experiment_std_05.31.2023.pkl']

In [8]:
output = pd.DataFrame(columns=['UACQR-P','UACQR-S', 'CQR','CQR-r', 'UACQR-P SE', 'UACQR-S SE', 'CQR SE','CQR-r SE'])
for i,result in enumerate(results):
    title = results[i].split('_')[0] if results[i].split('_')[0] not in ["facebook","meps"] else '_'.join(results[i].split('_')[0:2])
    result = pd.read_pickle('results/'+results[i])
    avg_result = result.loc[result.params.str.contains("'epoch_model_tracking': True")].groupby('params', sort=False).mean().iloc[:25]
    output.loc[title,['UACQR-P','UACQR-S', 'CQR','CQR-r']] = avg_result.min().to_numpy()
    output.loc[title,['UACQR-P SE','UACQR-S SE', 'CQR SE','CQR-r SE']] = result.groupby('params').sem().stack().loc[pd.MultiIndex.from_frame(avg_result.idxmin().rename('min_samples_leaf').reset_index().iloc[:,::-1])].to_numpy()
    
    

In [9]:
output.sort_index()

,UACQR-P,UACQR-S,CQR,CQR-r,UACQR-P SE,UACQR-S SE,CQR SE,CQR-r SE
bike,0.85401,1.01144,1.036681,0.99737,0.004511,0.018289,0.017872,0.018895
bio,1.566304,1.600183,1.588049,1.590436,0.004781,0.007891,0.005617,0.006593
blog,1.602354,1.920735,1.914235,2.146209,0.037509,0.049816,0.050879,0.107847
community,1.49244,1.603385,1.625074,1.626155,0.023068,0.024537,0.030009,0.027034
concrete,1.46543,1.496595,1.496024,1.496452,0.009443,0.011182,0.009727,0.009753
facebook_1,1.463887,1.951647,1.958679,2.061826,0.035439,0.099607,0.096426,0.145179
facebook_2,1.515316,2.00825,2.004542,2.044225,0.040791,0.050788,0.050715,0.070392
homes,0.551692,0.601959,0.601937,0.603195,0.003429,0.005988,0.00323,0.006536
meps_19,2.225645,2.512314,2.479244,2.483874,0.020481,0.034898,0.029183,0.036165
meps_20,2.189202,2.423887,2.407364,2.395958,0.019427,0.023738,0.025373,0.031972


In [10]:
output.drop(['blog','facebook_1','facebook_2','meps_19','meps_20','meps_21'],inplace=True)

In [11]:
output.sort_index()

,UACQR-P,UACQR-S,CQR,CQR-r,UACQR-P SE,UACQR-S SE,CQR SE,CQR-r SE
bike,0.85401,1.01144,1.036681,0.99737,0.004511,0.018289,0.017872,0.018895
bio,1.566304,1.600183,1.588049,1.590436,0.004781,0.007891,0.005617,0.006593
community,1.49244,1.603385,1.625074,1.626155,0.023068,0.024537,0.030009,0.027034
concrete,1.46543,1.496595,1.496024,1.496452,0.009443,0.011182,0.009727,0.009753
homes,0.551692,0.601959,0.601937,0.603195,0.003429,0.005988,0.00323,0.006536
star,0.178662,0.186614,0.186988,0.1892,0.000901,0.001292,0.001312,0.001488


In [12]:
def format_fig_and_se(row, method='CQR'):
    fig = row[method]
    se = row[method+' SE']
    if fig >1000:
        return f"{fig:.0f} ({se:.0f})"
    elif fig >1:
        return f"{fig:.3f} ({se:.3f})"
    else:
        return f"{fig:.3f} ({se:.3f})"

In [13]:
drop_columns = [c for c in output.columns if 'SE' in c]

In [14]:
from functools import partial

In [15]:
output['CQR'] = output.apply(partial(format_fig_and_se, method='CQR'), axis=1)
output['UACQR-S'] = output.apply(partial(format_fig_and_se, method='UACQR-S'), axis=1)
output['UACQR-P'] = output.apply(partial(format_fig_and_se, method='UACQR-P'), axis=1)
output['CQR-r'] = output.apply(partial(format_fig_and_se, method='CQR-r'), axis=1)

In [16]:
output.drop(columns=drop_columns, inplace=True)

In [17]:
# Export to LaTeX
latex_table = output.sort_index().style.to_latex()


print('\_'.join(latex_table.split('_')))

\begin{tabular}{lllll}
 & UACQR-P & UACQR-S & CQR & CQR-r \\
bike & 0.854 (0.005) & 1.011 (0.018) & 1.037 (0.018) & 0.997 (0.019) \\
bio & 1.566 (0.005) & 1.600 (0.008) & 1.588 (0.006) & 1.590 (0.007) \\
community & 1.492 (0.023) & 1.603 (0.025) & 1.625 (0.030) & 1.626 (0.027) \\
concrete & 1.465 (0.009) & 1.497 (0.011) & 1.496 (0.010) & 1.496 (0.010) \\
homes & 0.552 (0.003) & 0.602 (0.006) & 0.602 (0.003) & 0.603 (0.007) \\
star & 0.179 (0.001) & 0.187 (0.001) & 0.187 (0.001) & 0.189 (0.001) \\
\end{tabular}

